# Assignment 11 – Part 2: LLM Q&A Evaluator (OpenAI)

In this notebook I build a Q&A evaluator using an LLM from OpenAI.  
The goal is to:

- Load a dataset of ML questions and reference answers.
- Let a student answer a question.
- Use an LLM to compare the student answer with the reference.
- Produce qualitative feedback and a numerical score (0–100).


In [ ]:
!pip install --quiet openai

import os
import json
import random
from pathlib import Path
import textwrap

from openai import OpenAI

In [ ]:
# ⚠️ Set your OpenAI API key here or via Colab's environment settings.
# Option A (simple but less secure): uncomment and paste your key
# os.environ["OPENAI_API_KEY"] = "sk-REPLACE_WITH_YOUR_KEY"

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

assert client.api_key not in (None, "", "sk-REPLACE_WITH_YOUR_KEY"), "Set your OPENAI_API_KEY environment variable or in the cell above!"
print("OpenAI client configured ✅")

In [ ]:
from google.colab import files

print("Please upload Q&A_db_practice.json")
uploaded = files.upload()
uploaded

In [ ]:
DATA_PATH = Path("Q&A_db_practice.json")  # adjust if the uploaded file has a different name

def load_qa(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"Loaded {len(data)} Q&A pairs")
    print("Example keys:", list(data[0].keys()))
    return data

qa_db = load_qa(DATA_PATH)

In [ ]:
example = qa_db[0]

# Adapt the keys below if your JSON uses different names
print("QUESTION:\n", example["question"], "\n")
print("REFERENCE ANSWER (truncated):\n", textwrap.shorten(example["answer"], width=300, placeholder="..."))

In [ ]:
def call_llm(prompt: str) -> str:
    """
    Calls OpenAI chat completion and returns the text content.
    Using a relatively small, cheap model for grading.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # change model if you prefer another
        messages=[
            {"role": "system", "content": "You are a fair and rigorous ML exam grader."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.2,
    )
    return response.choices[0].message.content

In [ ]:
EVAL_INSTRUCTIONS = """
You are grading a student's answer to a machine learning theory question.

You will be given:
- The question
- A reference answer written by the instructor
- The student's answer

Tasks:
1. Compare the student's answer with the reference.
2. Comment on correctness, completeness, clarity, and missing points.
3. Give an integer score from 0 to 100, where:
   - 0–39: mostly incorrect or irrelevant
   - 40–69: partially correct with important gaps
   - 70–89: mostly correct with minor omissions
   - 90–100: very complete and accurate.

Respond ONLY in valid JSON with keys:
- "score": integer 0–100
- "feedback": short paragraph explanation.
"""

def evaluate_answer(question: str, ref_answer: str, student_answer: str) -> dict:
    prompt = f"""{EVAL_INSTRUCTIONS}

Question:
{question}

Reference answer:
{ref_answer}

Student answer:
{student_answer}

Now output the JSON:"""

    raw = call_llm(prompt).strip()

    # Handle ```json ... ``` wrappers if the model adds them
    if raw.startswith("```"):
        raw = raw.strip("`")
        if raw.lower().startswith("json"):
            raw = raw[4:]
        raw = raw.strip()

    try:
        result = json.loads(raw)
    except Exception:
        result = {
            "score": 0,
            "feedback": "Could not parse JSON. Raw model output:\n" + raw
        }
    return result

In [ ]:
sample = random.choice(qa_db)

q = sample["question"]
ref = sample["answer"]

print("QUESTION:\n", q, "\n")
print("REFERENCE ANSWER (truncated):\n", textwrap.shorten(ref, width=350, placeholder="..."), "\n")

# 👉 Replace this with your own attempt when testing
student_answer = """
Here I would write my own explanation of the concept, using my own words.
"""

result = evaluate_answer(q, ref, student_answer)
print("SCORE:", result["score"])
print("\nFEEDBACK:\n", result["feedback"])

In [ ]:
from difflib import SequenceMatcher

def text_similarity(a: str, b: str) -> float:
    """Very simple lexical similarity as a baseline (0–100)."""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() * 100

sim_score = text_similarity(student_answer, ref)
print(f"Naive similarity score (SequenceMatcher): {sim_score:.2f} / 100")

## Discussion of the approach

In this part of the assignment, I implemented an LLM-based evaluator for open-ended
ML theory questions. The pipeline is:

1. Load a JSON file (`Q&A_db_practice.json`) with questions and reference answers.
2. Let the student provide a free-text answer for a selected question.
3. Send the question, reference answer and student answer to an OpenAI model
   (`gpt-4o-mini`) with a carefully designed grading prompt.
4. The model returns a JSON object with:
   - a numerical score (0–100)
   - an explanatory feedback paragraph.
5. Optionally, I also compute a simple lexical similarity score between the
   student and reference answers using `difflib.SequenceMatcher` as a baseline.

This approach offloads the semantic comparison to the LLM, which can evaluate
correctness and completeness beyond exact word overlap. In a real system, the
LLM-based score and the lexical metric could be combined, and the collected
scores and feedback could later be used for model fine-tuning or RLHF-style
improvements.
